In [1]:
# NOTE: This is a very time consuming piece of code. When extracting audio features for all songs in the 64 years
# of the Eurovision Song Contest, it could take at least a week when only running the code below.
# I suggest doing it concurrent, under the first piece of codes is code for doing multiple years in parallel, 
# or 1 year in parallel, or one song.

from essentia import *
from essentia.standard import *
import essentia as es
import sys
import os
import json
import numpy as np
import matplotlib.pyplot as plt
import multiprocessing as mp

In [26]:
sbicPath = "example/SBiCs.sig"
with open(sbicPath) as f:
    segments = json.load(f)

In [4]:
# The extractedframe directory structure is the same as MP3. Per year there is a file for every segment in every 
# song.
extractedFramesPath = "example/extractedFrames"
mp3Path = "example/MP3"
yearDirectories = os.listdir(mp3Path)
yearDirectories = [dirs for dirs in yearDirectories if dirs[0] != "."]

In [41]:
# ALSO NOTE! This code is useless to run if the target directories do'nt exist!
# Run this for making these. (Gives error if the directories do already exist.)
for year in yearDirectories:
    os.mkdir(extractedFramesPath+"/"+year)

In [6]:
# Loop over year directories.
for year in yearDirectories:
#     Get mp3 files of year
        mp3s = os.listdir(mp3Path + "/" + str(year))
        mp3s = [mp3 for mp3 in mp3s if mp3[0] != "_"]
#         Loop over mp3 files
        for mp3 in mp3s:
#         Get country name for naming resulting file.
            country = mp3.split("_")[0]
#        Getting timestamps from SBiC files. It may go wrong that naming of mp3 file and SBiC is not the same.
            try :
                timestamps = segments[str(year)+"_"+country]["segmentation"]["timestamps"]
            except:
                print("Something went wrong when getting segments for: ", year, country)
                continue
            mp3name = mp3Path + "/" + str(year)+"/" + mp3
#             Loop over timestamps of song.
            for i in range(1, len(timestamps)):
                filename = extractedFramesPath + "/"+str(year)+"/"+str(year)+ "_"+country+str(i)+".json"
#             Extract audio features form segment and store in file.
                try:
                    features, features_frames = MusicExtractor(lowlevelStats=['mean', 'stdev', 'min', 'max'],
                                                      rhythmStats=['mean', 'stdev', 'min', 'max'],
                                                      tonalStats=['mean', 'stdev', 'min', 'max'],
                                                  startTime=timestamps[i-1],
                                                  endTime=timestamps[i])(mp3name)
                    YamlOutput(filename=filename, format="json")(features)
                except:
                    print("Something went wrong when extracting audio features for: ",  year, country, i)
        
            print("Done: ", year, country)
        print(year, "Is completed.")

Done:  2019 Netherlands
2019 Is completed.


In [22]:
# Code for running more years in parallel.
# Warning: Doing all 64 years may still take multiple days!

# # Get meta
# numberOfCores = 6
# # Divide year directories over cores.
# coreDivision = np.array_split(np.array(yearDirectories), numberOfCores)
# # Start processes
# processen = [mp.Process(target=frameExtractionParallelPerYear, 
#                         args=([coreDivision[x]])) for x in range(numberOfCores)]
# for p in processen:
#     p.start()
# # Always join processes.
# for p in processen:
#     p.join()

In [ ]:
def frameExtractionParallelPerYear(yearsInCore):
#     Loop over years from input.
    for year in yearsInCore:
#         From here is the same as above piece of code.
        mp3s = os.listdir(mp3Path + "/" + str(year))
        mp3s = [mp3 for mp3 in mp3s if mp3[0] != "_"]
        for mp3 in mp3s:
            country = mp3.split("_")[0]
            try :
                timestamps = segments[str(year)+"_"+country]["segmentation"]["timestamps"]
            except:
                print("Something went wrong when getting segments for: ", year, country)
                continue
            mp3name = mp3Path + "/" + str(year)+"/" + mp3
            for i in range(1, len(timestamps)):
                filename = extractedFramesPath + "/"+str(year)+"/"+str(year)+ "_"+country+str(i)+".json"
                try:
                    features, features_frames = MusicExtractor(lowlevelStats=['mean', 'stdev', 'min', 'max'],
                                                      rhythmStats=['mean', 'stdev', 'min', 'max'],
                                                      tonalStats=['mean', 'stdev', 'min', 'max'],
                                                  startTime=timestamps[i-1],
                                                  endTime=timestamps[i])(mp3name)
                    YamlOutput(filename=filename, format="json")(features)
                except:
                    print("Something went wrong when extracting audio features for: ",  year, country, i)
        
            print("Done: ", year, country)
        print(year, "Is completed.")

In [ ]:
# Code for running all songs in one year in parallel.

# # Get meta
# numberOfCores = 6
# yearToDoInParallel = "2019"
# # Get mp3 files for given year
# mp3s = os.listdir(mp3Path + "/" + str(yearToDoInParallel))
# mp3s = [mp3 for mp3 in mp3s if mp3[0] != "_"]
# # Divide mp3 files over cores.
# coreDivision = np.array_split(np.array(mp3s), numberOfCores)
# # Start processes
# processen = [mp.Process(target=frameExtractionParallelOneYear, 
#                         args=([coreDivision[x]])) for x in range(numberOfCores)]
# for p in processen:
#     p.start()
# # Always join processes
# for p in processen:
#     p.join()

In [23]:
def frameExtractionParallelOneYear(mp3sInCore):
#     Loop over inputted mp3 files
    for mp3 in mp3sInCore:
#         Code same as above. Only yearToDoInParallel is static.
        country = mp3.split("_")[0]
        try :
            timestamps = segments[str(yearToDoInParallel)+"_"+country]["segmentation"]["timestamps"]
        except:
            print("Something went wrong when getting segments for: ", yearToDoInParallel, country)
            continue
        mp3name = mp3Path + "/" + str(yearToDoInParallel)+"/" + mp3
        for i in range(1, len(timestamps)):
            filename = extractedFramesPath + "/"+str(yearToDoInParallel)+"/"+str(yearToDoInParallel)+ "_"+country+str(i)+".json"
            try:
                features, features_frames = MusicExtractor(lowlevelStats=['mean', 'stdev', 'min', 'max'],
                                                  rhythmStats=['mean', 'stdev', 'min', 'max'],
                                                  tonalStats=['mean', 'stdev', 'min', 'max'],
                                              startTime=timestamps[i-1],
                                              endTime=timestamps[i])(mp3name)
                YamlOutput(filename=filename, format="json")(features)
            except:
                print("Something went wrong when extracting audio features for: ",  yearToDoInParallel, country, i)

        print("Done: ", year, country)

In [39]:
# Code for running segments of one song in parallel.

# # Get meta
# numberOfCores = 6
# yearToDoInParallel = "2019"
# countryToDoInParallel = "Netherlands"
# songToDoInParallel = "Netherlands_Arcade_Duncan Laurence.mp3"
# mp3ToDoInParallel = "example/MP3/"+yearToDoInParallel+"/"+songToDoInParallel

# # Get the timestamps
# timestamps = segments[str(yearToDoInParallel)+"_"+countryToDoInParallel]["segmentation"]["timestamps"]

# # Divide timestamps over the cores
# segmentsForParallel = []
# for i in range(1, len(timestamps)):
#     segmentsForParallel.append((i, timestamps[i-1], timestamps[i]))
    
# coreDivision = np.array_split(np.array(segmentsForParallel), numberOfCores)

# # Start processes.
# processen = [mp.Process(target=frameExtractionParallelOneSong, 
#                         args=([coreDivision[x]])) for x in range(numberOfCores)]
# for p in processen:
#     p.start()
# # Always join processes.
# for p in processen:
#     p.join()

In [40]:
def frameExtractionParallelOneSong(segmentsInCore):
#     Loop over inputted segments of song to choice.
    for i, start, end in segmentsInCore:
#         Make filename with above defined year and country
        filename = extractedFramesPath + "/"+str(yearToDoInParallel)+"/"+str(yearToDoInParallel)+ "_"+countryToDoInParallel+str(i)+".json"
        try:
            features, features_frames = MusicExtractor(lowlevelStats=['mean', 'stdev', 'min', 'max'],
                                              rhythmStats=['mean', 'stdev', 'min', 'max'],
                                              tonalStats=['mean', 'stdev', 'min', 'max'],
                                          startTime=start,
                                          endTime=end)(mp3ToDoInParallel)
            YamlOutput(filename=filename, format="json")(features)
            print("Finished segment: ", i)
        except:
            print("Something went wrong when extracting audio features for: ",  year, country, i)
